# Daub 5/3 integer-to-integer system

In [1]:
import numpy as np
import matplotlib.pyplot as plt

Импорт пользовательских функций.

In [2]:
from Functions.daub_5_3 import daub_5_3

In [3]:
#f = [1, -2, 6, 8]
f = np.random.randint(low=-10, high=11, size=8)
N = len(f)
d = [0] * (N//2)
a = [0] * (N//2)

Функция преобразования индексов по правилу "even symmetry".

In [4]:
def indx_even(k,N):
    reminder = abs(k) % (2*(N-1))
    if reminder <= (N-1):
        # rise
        return reminder
    else:
        # fall
        return 2*(N-1) - reminder

## Прямое преобразование

Функция вычисления коэффициентов ${d_k}$ (флуктуации).

In [5]:
def d_func(k,f,N):
    return f[indx_even(2*k+1,N)] - 1/2*(f[indx_even(2*k,N)] + f[indx_even(2*k+2,N)])

In [6]:
for k in range(N//2):
    d[k] = d_func(k,f,N)
    a[k] = f[indx_even(2*k,N)] + 1/4*(d_func(k-1,f,N) + d[k])
    #a[k] = -1/8*(f[indx_even(2*k-2,N)] + f[indx_even(2*k+2,N)]) + 1/4*(f[indx_even(2*k-1,N)] + f[indx_even(2*k+1,N)]) + 3/4*f[indx_even(2*k,N)]
print('a', a)
print('d', d)

a [-12.5, 5.625, 6.5, -0.875]
d [-7.0, -6.5, 8.5, 16.0]


## Обратное преобразование

In [7]:
f_rec = [0] * N
# Odd values.
for k in range(N//2):
    f_rec[2*k] = a[k] - 1/4*(d_func(k-1,f,N) + d[k])

# Even values.
for k in range(N//2):
    f_rec[2*k+1] = d[k] + 1/2*(f[2*k] + f[indx_even(2*k+2,N)])
print(f)
print(f_rec)

[-9 -7  9  1  6  8 -7  9]
[-9.0, -7.0, 9.0, 1.0, 6.0, 8.0, -7.0, 9.0]


## Прямое и обратное преобразование при помощи матричных операций

In [8]:
Mdir = np.matrix(np.concatenate(daub_5_3(N), axis = 0))
tr = np.matrix(f)*(Mdir.T)
Mrev = np.linalg.inv(Mdir).T
print('Wavelet coefficients:', tr.tolist()[0])
print('Reconstruction:', (tr * Mrev).tolist()[0])
print('Original:', f)

Wavelet coefficients: [-12.5, 5.625, 6.5, -0.875, -7.0, -6.5, 8.5, 16.0]
Reconstruction: [-9.0, -7.0, 9.0, 1.0, 6.0, 8.0, -7.0, 9.0]
Original: [-9 -7  9  1  6  8 -7  9]


In [9]:
np.round(Mdir)

matrix([[ 1.,  0., -0.,  0.,  0.,  0.,  0.,  0.],
        [-0.,  0.,  1.,  0., -0.,  0.,  0.,  0.],
        [ 0.,  0., -0.,  0.,  1.,  0., -0.,  0.],
        [ 0.,  0.,  0.,  0., -0.,  0.,  1.,  0.],
        [-0.,  1., -0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0., -0.,  1., -0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0., -0.,  1., -0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0., -1.,  1.]])